In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time
import functools
from queue import PriorityQueue

#from ugs import *
# Need to make sure it's installed: pip3 install line_profiler
%load_ext line_profiler

In [2]:
@functools.lru_cache(maxsize=32768)
def distance(current, _next):
    # from https://stackoverflow.com/a/1401828
    if current == _next:
        return 0
    return int(np.linalg.norm(mesh.points[current]-mesh.points[_next]))

@functools.lru_cache(maxsize=183746)
def graph_neighbours(current):
    # Get the points from the cells which have the same point in them
    points = np.unique(mesh.cells['triangle'][np.where(mesh.cells['triangle']==current)[0]])
    # remove the current point from these results:
    points = points[points != current]
    
    # Get the elevation of those connected points
    elevations = mesh.point_data['Z'][points]
    # Return a list of connected points, as long as they are above sea-level
    return points[elevations >= 0]
    

def cost_search(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_dist < max_distance:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far

def cost_search(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_cost <= max_fuel:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far, dist_so_far

def get_total_cost_for_point(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_distance, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_cost = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_cost += cost_so_far[k]
            
    return total_cost

def get_total_distance_for_all_paths_to_point(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_dist = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_dist += dist_so_far[k]
            
    return total_dist

def get_from_cell(cell, travel_cost_function, max_distance=100000):
    start = cell[1]
    return get_from_point(start, travel_cost_function, max_distance)

def get_dist_from_point(point, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_dist = get_total_distance_for_all_paths_to_point(point, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_dist)

def get_from_point(point, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_cost = get_total_cost_for_point(point, travel_cost_function, max_distance, graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_cost)

In [3]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def distance_with_elevation_scaled(current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

In [4]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def elevation_only(current, _next):
    # Only take into account elevation changes for costs
    if current == _next:
        return 0
    return int(abs(mesh.point_data['Z'][current] - mesh.point_data['Z'][_next]) + distance(current, _next)*0.005)

In [5]:
infile = "australia/data/AUS.vtk"
outfile = "costed_aus_new.vtk"
mesh = meshio.read(infile)

max_fuel = 1500
                      # A smaller value means visiting far fewer nodes, so it speeds things up a lot

In [6]:
# Choose which cost function you want
travel_cost = elevation_only

## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [7]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [8]:
print("Total starting points available: ", points_above_sealevel.shape[0])

Total starting points available:  138726


## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [9]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_dist_from_point, travel_cost_function = travel_cost, max_fuel = max_fuel)

In [10]:
#%lprun -f get_from_point_in_mesh get_from_point_in_mesh(points_above_sealevel[100])

In [11]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print(travel_cost(points_above_sealevel[100], points_above_sealevel[101]))
print(get_from_point_in_mesh(points_above_sealevel[100]))
print(graph_neighbours.cache_info())

2938
(183253, 134578594)
CacheInfo(hits=58824, misses=3045, maxsize=183746, currsize=3045)


In [12]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

p = Pool(6)
start = 0
inc = 500
stop = inc

while start < points_above_sealevel.shape[0]-1:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print("From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0]))
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(outfile, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1

From  0  to  500  took  116.03642845153809 seconds, starting with point 183353 . Percent complete:  0.3604227037469544
From  500  to  1000  took  77.11723923683167 seconds, starting with point 182853 . Percent complete:  0.7208454074939088
From  1000  to  1500  took  72.68580889701843 seconds, starting with point 182353 . Percent complete:  1.0812681112408633
From  1500  to  2000  took  75.96782803535461 seconds, starting with point 181853 . Percent complete:  1.4416908149878176
From  2000  to  2500  took  78.21221160888672 seconds, starting with point 181353 . Percent complete:  1.802113518734772
From  2500  to  3000  took  77.4963755607605 seconds, starting with point 180853 . Percent complete:  2.1625362224817266
From  3000  to  3500  took  367.5736804008484 seconds, starting with point 180353 . Percent complete:  2.522958926228681
From  3500  to  4000  took  307.77855706214905 seconds, starting with point 179787 . Percent complete:  2.883381629975635
From  4000  to  4500  took  175

From  33500  to  34000  took  120.23774123191833 seconds, starting with point 140361 . Percent complete:  24.5087438547929
From  34000  to  34500  took  126.92316579818726 seconds, starting with point 139711 . Percent complete:  24.869166558539856
From  34500  to  35000  took  124.41379570960999 seconds, starting with point 139039 . Percent complete:  25.22958926228681
From  35000  to  35500  took  117.82445073127747 seconds, starting with point 138381 . Percent complete:  25.590011966033764
From  35500  to  36000  took  122.15948939323425 seconds, starting with point 137718 . Percent complete:  25.95043466978072
From  36000  to  36500  took  117.96129083633423 seconds, starting with point 137056 . Percent complete:  26.310857373527675
From  36500  to  37000  took  126.13439798355103 seconds, starting with point 136404 . Percent complete:  26.671280077274627
From  37000  to  37500  took  137.22139310836792 seconds, starting with point 135764 . Percent complete:  27.031702781021583
From

From  67500  to  68000  took  110.48212242126465 seconds, starting with point 95076 . Percent complete:  49.0174877095858
From  68000  to  68500  took  132.40651035308838 seconds, starting with point 94404 . Percent complete:  49.377910413332756
From  68500  to  69000  took  125.29912662506104 seconds, starting with point 93753 . Percent complete:  49.73833311707971
From  69000  to  69500  took  131.6738784313202 seconds, starting with point 93056 . Percent complete:  50.09875582082667
From  69500  to  70000  took  119.6860876083374 seconds, starting with point 92429 . Percent complete:  50.45917852457362
From  70000  to  70500  took  115.87567567825317 seconds, starting with point 91761 . Percent complete:  50.81960122832058
From  70500  to  71000  took  121.69624090194702 seconds, starting with point 91103 . Percent complete:  51.18002393206753
From  71000  to  71500  took  131.59797716140747 seconds, starting with point 90439 . Percent complete:  51.54044663581448
From  71500  to  7

From  101000  to  101500  took  126.64636707305908 seconds, starting with point 51005 . Percent complete:  73.16580886063176
From  101500  to  102000  took  124.89168167114258 seconds, starting with point 50345 . Percent complete:  73.5262315643787
From  102000  to  102500  took  118.00845289230347 seconds, starting with point 49689 . Percent complete:  73.88665426812567
From  102500  to  103000  took  131.9352102279663 seconds, starting with point 49011 . Percent complete:  74.24707697187262
From  103000  to  103500  took  122.06225609779358 seconds, starting with point 48351 . Percent complete:  74.60749967561956
From  103500  to  104000  took  137.12921929359436 seconds, starting with point 47694 . Percent complete:  74.96792237936653
From  104000  to  104500  took  135.3139786720276 seconds, starting with point 47032 . Percent complete:  75.32834508311348
From  104500  to  105000  took  128.73770952224731 seconds, starting with point 46371 . Percent complete:  75.68876778686044
Fro

From  134000  to  134500  took  125.12806844711304 seconds, starting with point 7455 . Percent complete:  96.95370730793074
From  134500  to  135000  took  123.28298211097717 seconds, starting with point 6822 . Percent complete:  97.31413001167769
From  135000  to  135500  took  129.2458119392395 seconds, starting with point 6152 . Percent complete:  97.67455271542464
From  135500  to  136000  took  122.1272280216217 seconds, starting with point 5483 . Percent complete:  98.0349754191716
From  136000  to  136500  took  128.85415530204773 seconds, starting with point 4829 . Percent complete:  98.39539812291855
From  136500  to  137000  took  116.87764477729797 seconds, starting with point 4174 . Percent complete:  98.75582082666551
From  137000  to  137500  took  132.25251579284668 seconds, starting with point 3539 . Percent complete:  99.11624353041246
From  137500  to  138000  took  119.9079110622406 seconds, starting with point 2870 . Percent complete:  99.47666623415942
From  138000

## Output

Show some of the results:

In [ ]:
print all_costs[:100]